# Chapter 15 : Design

Since this is about design, then it's probably the most important chapter, right?



In [1]:
#include <algorithm>
#include <iostream>
#include <cstddef>
#include <stdexcept>

In [2]:
.rawInput

Using raw input


In [3]:
class Ref_count {
public:
  Ref_count(): count(new std::size_t(1)) { }

  std::size_t val() const { return *count; }
  void operator ++() const { ++*count; }
  void operator --() const { if (--*count == 0) delete count; }

private:
  std::size_t* count;
};

template <class T> class Ptr {
 public:
  void make_unique() {
    if (refptr.val() != 1) {
      --refptr;
      refptr = Ref_count();
      p = p? clone(p): 0;
    }
  }

 Ptr(): p(0) { }
 Ptr(T* t): p(t) { }
 Ptr(const Ptr& h): p(h.p), refptr(h.refptr) { ++refptr; }

  Ptr& operator=(const Ptr&);
  ~Ptr();
  operator bool() const { return p; }
  T& operator*() const;
  T* operator->() const;

 private:
  T* p;
  Ref_count refptr;
};

template <class T> T* clone(const T* tp) {
  return tp->clone();
}

template<class T> T& Ptr<T>::operator*() const { 
  if (p) return *p; throw std::runtime_error("unbound Ptr"); 
}

template<class T> T* Ptr<T>::operator->() const { 
  if (p) return p; throw std::runtime_error("unbound Ptr"); 
}

template<class T> Ptr<T>& Ptr<T>::operator=(const Ptr& rhs) {
  ++rhs.refptr;
  --refptr;

  if (refptr.val() == 0) {
    delete p;
  }

  refptr = rhs.refptr;
  p = rhs.p;

  return *this;
}

  template<class T> Ptr<T>::~Ptr() {
    --refptr;

    if (refptr.val() == 0) {
      delete p;
    }
}

In [4]:
.rawInput

Not using raw input


In [5]:
.rawInput

Using raw input


In [6]:
class Pic_base {
  friend std::ostream& operator<<(std::ostream&, const Picture&);
  friend class Frame_Pic;
  friend class HCat_Pic;
  friend class VCat_Pic;
  friend class String_Pic;
  friend class Picture;

  // no `public' interface (except for the destructor)
  typedef Vec<Str>::size_type ht_sz;
  typedef Str::size_type wd_sz;

  // this class is an abstract base class
  virtual wd_sz width() const = 0;
  virtual ht_sz height() const = 0;
  virtual void display(std::ostream&, ht_sz, bool) const = 0;
  virtual void reframe(char, char, char) = 0;

 public:
  virtual ~Pic_base() { }

 protected:
  static void pad(std::ostream& os, wd_sz, wd_sz);
};

class Picture {
  friend std::ostream& operator<<(std::ostream&, const Picture&);
  friend Picture frame(const Picture&);
  friend Picture hcat(const Picture&, const Picture&);
  friend Picture vcat(const Picture&, const Picture&);

 public:
  Picture(const Vec<Str>& = Vec<Str>());
  void reframe(char c, char t_b, char s) { p->reframe(c, t_b, s); }

 private:
 Picture(Pic_base* ptr): p(ptr) { }
  Ptr<Pic_base> p;
};

input_line_5:2:56: error: unknown type name 'Picture'
  friend std::ostream& operator<<(std::ostream&, const Picture&);
                                                       ^
input_line_5:10:11: error: no template named 'Vec'
  typedef Vec<Str>::size_type ht_sz;
          ^
input_line_5:10:15: error: use of undeclared identifier 'Str'; did you mean 'Ptr'?
  typedef Vec<Str>::size_type ht_sz;
              ^~~
              Ptr
input_line_4:13:26: note: 'Ptr' declared here
template <class T> class Ptr {
                         ^
input_line_5:10:15: error: no template named 'Str'; did you mean 'Ptr'?
  typedef Vec<Str>::size_type ht_sz;
              ^~~
              Ptr
input_line_4:13:26: note: 'Ptr' declared here
template <class T> class Ptr {
                         ^
input_line_5:10:21: error: non-friend class member 'size_type' cannot have a qualified name
  typedef Vec<Str>::size_type ht_sz;
                  ~~^
input_line_5:10:21: error: typedef declarator cannot be qualifi

ename: evalue

In [ ]:
.rawInput

In [ ]:
.rawInput

In [ ]:
Picture frame(const Picture&);
Picture hcat(const Picture&, const Picture&);
Picture vcat(const Picture&, const Picture&);
std::ostream& operator<<(std::ostream&, const Picture&);

class String_Pic: public Pic_base {
  friend class Picture;
  Vec<Str> data;
 String_Pic(const Vec<Str>& v): data(v) { }

  ht_sz height() const { return data.size(); }
  wd_sz width() const;
  void display(std::ostream&, ht_sz, bool) const;
  void reframe(char, char, char) { }
};

class VCat_Pic: public Pic_base {
  friend Picture vcat(const Picture&, const Picture&);

  Ptr<Pic_base> top, bottom;

 VCat_Pic(const Ptr<Pic_base>& t, const Ptr<Pic_base>& b):
  top(t), bottom(b) { }

  wd_sz width() const { return std::max(top->width(), bottom->width()); }
  ht_sz height() const { return top->height() + bottom->height(); }
  void display(std::ostream&, ht_sz, bool) const;
  void reframe(char, char, char);
};

class HCat_Pic: public Pic_base {
  friend Picture hcat(const Picture&, const Picture&);

  Ptr<Pic_base> left, right;

 HCat_Pic(const Ptr<Pic_base>& l, const Ptr<Pic_base>& r):
  left(l), right(r) { }

  wd_sz width() const { return left->width() + right->width(); }
  ht_sz height() const { return std::max(left->height(), right->height()); }
  void display(std::ostream&, ht_sz, bool) const;
  void reframe(char, char, char);
};

class Frame_Pic: public Pic_base {
  friend Picture frame(const Picture&);

  Ptr<Pic_base> p;
  char corner;
  char top_bottom;
  char side;

 Frame_Pic(const Ptr<Pic_base>& pic): p(pic), corner('*'), top_bottom('-'), 
    side('|') { }

  wd_sz width() const { return p->width() + 4; }
  ht_sz height() const { return p->height() + 4; }
  void display(std::ostream&, ht_sz, bool) const;
  void reframe(char, char, char);
};

In [ ]:
.rawInput